Implementing  Sequential Agents

# Onboarding chats

Use Case: In this project i will set up agents that can be used to onboard users into a live chat system like the one you can find on an online app like your banking website or your phone provider website. I will set up an agents to gather specifc information about a user that needs help, and then pass that information to human that will directly have a clear understanding of who requires help, where they're from and what their problem is, so that the human can immediately act on this information and help the user.

To do so,I am  going to create 3 agents:
1. An **Onboarding Personal Information Agent**: this agent's goal is to get the name and location of the customer.
2. An **Onboarding Issue Agent**: this agent's goal is to determine what the issue of the customer is.
3. A  **Customer Engagement Agent**: this agent will interact with the customer until a human agent can start interacting with the customer.

In [2]:
import os 
from dotenv import load_dotenv
from autogen import ConversableAgent
from autogen import initiate_chats

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
#llm configuration
llm_config = {
    "model":"gpt-4o-mini",
    "api_key":api_key
}

## Agents definition

- Onboarding Personal Information Agent 1 : Goal to get customer's name and location 

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you work for a phone provider called ACME.
    Your job is to gather the customer's name and location.
    Do not ask for any other information, only ask about the customer's name and location.
    After the customer gives you their name and location, repeat them 
    and thank the user, and ask the user to answer with TERMINATE to move on to describing their issue.
    ''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

- Onboarding Issue Agent 2 : Goal to get customer product use info and problem facing 

In [5]:
onboarding_issue_agent = ConversableAgent(
    name="Onboarding_Issue_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you work for a phone provider called ACME,
    you are here to help new customers get started with our product.
    Your job is to gather the product the customer use and the issue they currently 
    have with the product,
    Do not ask for other information.
    After the customer describes their issue, repeat it and add
    "Please answer with 'TERMINATE' if I have correctly understood your issue." ''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower()
)

- Customer Engagement Agent 3: goal to keep customer entertained

In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''You are a helpful customer service agent.
    Your job is to gather customer's preferences on news topics.
    You are here to provide fun and useful information to the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

- Adding myself as a customer : Goal talk about my issue with the product

In [7]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

## Chat orchestration

- Here there will be 3 chats that will take place in one loop. Agent1:User, Agent2:USer, Agent3:User. So will capture that inform a of a list.

### 1. Onboarding Agent with Customer

In [9]:
chats = []  # empty list to hold all conversations

- the first conversation that takes place will be sent to the next agent and so that they have context about previous conversation. same way agent 2 and user chat will be sent to next agent.and so on so we have data in the form of dict,

In [10]:
chats.append(
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you solve any issue you have with our products. "
            "Could you tell me your name?",  #starter msg
        "summary_method": "reflection_with_llm", #analyse the summary dict
        "summary_args": {
        "summary_prompt" : "Return the customer information "
                             "into a JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "clear_history" : True
    }
)

### 2. Issue Agent with Customer

In [11]:
chats.append(
        {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "While we're waiting for a human agent to take over and help you solve "
        "your issue, can you tell me more about how you use our products or some "
        "topics interesting for you?", #starter msg
        "max_turns": 2,
        "summary_method": "reflection_with_llm",
    }
)
chats

[{'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a56c9af0>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a43fef30>,
  'message': "Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?",
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return the customer information into a JSON object only: {'name': '', 'location': ''}"},
  'clear_history': True},
 {'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a43fef30>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a6c37b00>,
  'message': "While we're waiting for a human agent to take over and help you solve your issue, can you tell me more about how you use our products or some topics interesting for you?",
  'max_turns': 2,
  'summary_method': 'reflection_with_llm'}]

### 3. Customer Engagement Agent with Customer

In [12]:
chats.append(
    {
        "sender": customer_proxy_agent,
        "recipient":customer_engagement_agent,
        "message":
                "While we're waiting for a human agent to take over and help you solve "
        "your issue, can you tell me more about how you use our products or some "
        "topics interesting for you?", #starter msg
        "max_turns": 2,
        "summary_method": "reflection_with_llm",
    }
)
chats

[{'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a56c9af0>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a43fef30>,
  'message': "Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?",
  'summary_method': 'reflection_with_llm',
  'summary_args': {'summary_prompt': "Return the customer information into a JSON object only: {'name': '', 'location': ''}"},
  'clear_history': True},
 {'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a43fef30>,
  'recipient': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a6c37b00>,
  'message': "While we're waiting for a human agent to take over and help you solve your issue, can you tell me more about how you use our products or some topics interesting for you?",
  'max_turns': 2,
  'summary_method': 'reflection_with_llm'},
 {'sender': <autogen.agentchat.conversable_agent.ConversableAgent at 0x214a43fef30>,
  'recip

## Initiate the sequential chat

In [13]:
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Hello, I'm here to help you solve any issue you have with our products. Could you tell me your name?

--------------------------------------------------------------------------------


c:\code\Projects\Agentic_AI_AutoGen\venv\Lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding_Personal_Information_Agent):



--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Thank you! Now, could you please provide me with your location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Ilyaas

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Thank you, Ilyaas! Now, could you please provide me with your location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Auston,Texas

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent 

In [14]:
import pprint

for chat_result in chat_results:
    #pprint.pprint(chat_result.chat_history) # We could also get the whole chat history with this command
    pprint.pprint(chat_result.summary)

'```json\n{\n  "name": "Ilyaas",\n  "location": "Auston, Texas"\n}\n```'
('Ilyaas is interested in Apple and its monthly EMI options. The conversation '
 "touched on Apple's financing solutions, like the Apple Card, offering lower "
 "interest rates and cash back. A fun fact about the original Apple computer's "
 'price being $666.66 was shared, along with an offer for more information or '
 'a tech joke.')
('Ilyaas is interested in dust storms and has been reading about them. The '
 'conversation included fascinating facts about dust storms, such as their '
 'occurrence during the Dust Bowl and their impact on ocean fertilization. A '
 'light-hearted joke related to weather was shared for entertainment.')


The human can thus quickly learn about our customer, their name, location and issue and immediately get started on solving the issue.

## What is this for?

This example shows you how you can integrate LLMs into an app in order to accomplish complex interactions with humans and gather specific information through natural communication. This is very powerful and is not something that was easy to do it before having LLMs. By having agents focused on a single task, we're able to ensure that the LLM won't diverge and remain focused on a simple task, greatly increasing chances that it accurately accomplishes it.